<a href="https://colab.research.google.com/github/gabcsx/CCMACLRL_EXAM/blob/main/Midterm_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import re

## Dataset File

In [173]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv(dataset_url)

In [174]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [176]:
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
ext_col,0
int_col,0


In [177]:
df.dropna(inplace=True)
df.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,0
engine,0
transmission,0
ext_col,0
int_col,0


In [178]:
df.describe()

,id,model_year,milage,price
count,162610.000000,162610.000000,162610.000000,1.626100e+05
mean,94197.039137,2015.158822,71105.487086,4.081699e+04
std,54467.770815,5.699531,50157.874111,7.653857e+04
min,0.000000,1974.000000,100.000000,2.000000e+03
25%,47026.250000,2012.000000,29000.000000,1.570000e+04
50%,94139.000000,2016.000000,64300.000000,2.850000e+04
75%,141421.750000,2019.000000,101000.000000,4.699900e+04
max,188532.000000,2024.000000,405000.000000,2.954083e+06


In [179]:
df.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
108016,108016,Mercedes-Benz,E-Class E 350 4MATIC,2016,47000,Gasoline,268.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,None reported,Yes,25900
157902,157902,Lexus,RX 350 Crafted Line F Sport,2016,159870,Gasoline,268.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,CVT Transmission,White,Black,At least 1 accident or damage reported,Yes,19950
174348,174348,Audi,S4 3.0T Premium Plus,2012,120000,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes,10000
22666,22666,Chevrolet,Silverado 1500 LT Extended Cab,2002,119000,Gasoline,285.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,4-Speed A/T,Red,–,At least 1 accident or damage reported,Yes,16500
51590,51590,RAM,1500 Limited,2022,22000,Hybrid,395.0HP 5.7L 8 Cylinder Engine Gasoline/Mild E...,8-Speed A/T,Gray,Black,None reported,Yes,64000
149218,149218,Lexus,RX 350 F Sport,2017,53649,Gasoline,295.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,At least 1 accident or damage reported,Yes,23500
121514,121514,Hyundai,Santa Fe Sport 2.0L Turbo Ultimate,2010,83280,Gasoline,242.0HP 3.3L V6 Cylinder Engine Gasoline Fuel,A/T,White,Gray,At least 1 accident or damage reported,Yes,10500
155153,155153,Jeep,Wrangler Unlimited Sahara,2019,57000,Gasoline,285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Gray,Gray,None reported,Yes,39000
119158,119158,Audi,Q5 2.0T Premium Plus,2019,50000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Beige,Black,At least 1 accident or damage reported,Yes,29999
114736,114736,Jeep,Wrangler Unlimited Rubicon,2017,54500,Gasoline,285.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,29995


In [180]:
current_year = datetime.now().year
df['car_age'] = current_year - df['model_year'].astype(int)

In [181]:
luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Tesla', 'Porsche', 'Land', 'Aston', 'Lexus', 'Lincoln', 'Lamborghini', 'Bugatti', 'Cadillac', 'Maserati', 'Genesis', ' Acura']
df['Luxurious'] = df['brand'].apply(lambda x: 'Yes' if x in luxury_brands else 'No')

In [182]:
label_encoder = LabelEncoder()
df['brand'] = label_encoder.fit_transform(df['brand'])
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])
df['transmission'] = label_encoder.fit_transform(df['transmission'])
df['ext_col'] = label_encoder.fit_transform(df['ext_col'])
df['int_col'] = label_encoder.fit_transform(df['int_col'])
df['accident'] = label_encoder.fit_transform(df['accident'])
df['clean_title'] = label_encoder.fit_transform(df['clean_title'])
df['Luxurious'] = label_encoder.fit_transform(df['Luxurious'])

In [183]:
def extract_engine_features(engine_desc):
    hp_match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    hp = float(hp_match.group(1)) if hp_match else None

    l_match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    displacement = float(l_match.group(1)) if l_match else None

    cyl_match = re.search(r'(\d+) Cylinder', engine_desc)
    cylinders = int(cyl_match.group(1)) if cyl_match else None

    turbocharged = 1 if 'Turbo' in engine_desc else 0

    return hp, displacement, cylinders, turbocharged

df[['HP', 'Liters', 'Cylinders', 'Turbocharged']] = df['engine'].apply(
    lambda x: pd.Series(extract_engine_features(x))
)
drop_columns = ['engine', 'model']
df.drop(drop_columns, axis=1, inplace=True)

df.fillna(df.mean(), inplace=True)

df.sample(15)

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,car_age,Luxurious,HP,Liters,Cylinders,Turbocharged
150775,150775,8,2017,159500,2,2,28,10,0,0,22500,7,1,420.0,6.2,8.0,0.0
129156,129156,43,2023,5400,2,23,298,14,1,0,76500,1,1,450.0,3.0,6.0,0.0
114495,114495,4,2011,111000,2,38,298,10,1,0,15990,13,1,300.0,3.0,6.0,0.0
147124,147124,3,2018,38718,2,31,28,14,1,0,32500,6,1,333.0,3.0,6.0,0.0
188497,188497,14,2021,24000,2,2,298,71,1,0,47950,3,0,375.0,3.5,6.0,0.0
36198,36198,8,2021,8497,2,2,28,14,1,0,52900,3,1,420.0,6.2,8.0,0.0
97588,97588,4,2014,73500,2,31,257,14,1,0,21500,10,1,300.0,3.0,6.0,0.0
80023,80023,27,2011,145000,2,38,120,67,0,0,14500,13,1,270.0,3.5,6.0,0.0
8720,8720,8,2023,1185,2,2,298,10,1,0,143900,1,1,420.0,6.2,8.0,0.0
98209,98209,27,2015,69423,2,16,28,10,0,0,21500,9,1,301.0,4.6,8.0,0.0


In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            162610 non-null  int64  
 1   brand         162610 non-null  int64  
 2   model_year    162610 non-null  int64  
 3   milage        162610 non-null  int64  
 4   fuel_type     162610 non-null  int64  
 5   transmission  162610 non-null  int64  
 6   ext_col       162610 non-null  int64  
 7   int_col       162610 non-null  int64  
 8   accident      162610 non-null  int64  
 9   clean_title   162610 non-null  int64  
 10  price         162610 non-null  int64  
 11  car_age       162610 non-null  int64  
 12  Luxurious     162610 non-null  int64  
 13  HP            162610 non-null  float64
 14  Liters        162610 non-null  float64
 15  Cylinders     162610 non-null  float64
 16  Turbocharged  162610 non-null  float64
dtypes: float64(4), int64(13)
memory usage: 22.3 MB


In [185]:
X = df.drop(['price'],axis=1)
y = df['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=46)

In [186]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

68355.7732177658

## Test File

In [187]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [188]:
dt.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [189]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [190]:
dt.isnull().sum()

,0
id,0
brand,0
model,0
model_year,0
milage,0
fuel_type,3383
engine,0
transmission,0
ext_col,0
int_col,0


In [191]:
dt.describe()

,id,model_year,milage
count,125690.000000,125690.000000,125690.000000
mean,251377.500000,2015.797526,66042.581510
std,36283.722005,5.673797,50223.858435
min,188533.000000,1974.000000,100.000000
25%,219955.250000,2013.000000,24500.000000
50%,251377.500000,2017.000000,57500.000000
75%,282799.750000,2020.000000,95798.000000
max,314222.000000,2024.000000,405000.000000


In [192]:
dt.sample(20)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
115491,304024,Toyota,Sequoia Limited,2017,67000,Gasoline,381.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes
98047,286580,RAM,1500 Big Horn,2020,35289,Gasoline,3.6L V6 24V MPFI DOHC,8-Speed Automatic,Bright White Clearcoat,Diesel Gray / Black,None reported,Yes
49730,238263,Kia,Optima Hybrid EX,2016,96419,Hybrid,199.0HP 2.4L 4 Cylinder Engine Gas/Electric Hy...,6-Speed A/T,Black,Beige,None reported,Yes
104141,292674,GMC,Sierra 2500 Denali,2021,63460,Diesel,445.0HP 6.6L 8 Cylinder Engine Diesel Fuel,10-Speed A/T,Gray,Beige,At least 1 accident or damage reported,Yes
23092,211625,Ford,Transit-350 XLT,2018,67500,Gasoline,310.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Black,At least 1 accident or damage reported,Yes
89113,277646,Nissan,Frontier SV,2011,96600,Gasoline,261.0HP 4.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Gray,At least 1 accident or damage reported,Yes
41711,230244,Mercedes-Benz,AMG C 43 Base 4MATIC,2018,7751,Gasoline,362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,9-Speed A/T,Black,Black,None reported,Yes
86469,275002,Land,Rover Range Rover Evoque SE Premium,2020,44457,Gasoline,2.0 Liter,Automatic,Black,Black,None reported,NaN
96120,284653,Ford,F-150 Raptor,2015,96479,Gasoline,6.2L V8 16V GDI OHV,10-Speed Automatic,Red,Black,None reported,Yes
96671,285204,Ford,Mustang GT Premium,2020,19200,Gasoline,460.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,10-Speed A/T,Gray,Black,None reported,Yes


In [193]:
current_year = datetime.now().year
dt['car_age'] = current_year - df['model_year'].astype(int)

In [194]:
luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Tesla', 'Porsche', 'Land', 'Aston', 'Lexus', 'Lincoln', 'Lamborghini', 'Bugatti', 'Cadillac', 'Maserati', 'Genesis', ' Acura']
dt['Luxurious'] = df['brand'].apply(lambda x: 'Yes' if x in luxury_brands else 'No')

In [195]:
label_encoder = LabelEncoder()
dt['brand'] = label_encoder.fit_transform(dt['brand'])
dt['fuel_type'] = label_encoder.fit_transform(dt['fuel_type'])
dt['transmission'] = label_encoder.fit_transform(dt['transmission'])
dt['ext_col'] = label_encoder.fit_transform(dt['ext_col'])
dt['int_col'] = label_encoder.fit_transform(dt['int_col'])
dt['accident'] = label_encoder.fit_transform(dt['accident'])
dt['clean_title'] = label_encoder.fit_transform(dt['clean_title'])
dt['Luxurious'] = label_encoder.fit_transform(dt['Luxurious'])

In [196]:
# Function to extract engine features
def extract_engine_features(engine_desc):
    # Extract horsepower (HP)
    hp_match = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    hp = float(hp_match.group(1)) if hp_match else None

    # Extract engine displacement (L)
    l_match = re.search(r'(\d+(\.\d+)?)L', engine_desc)
    displacement = float(l_match.group(1)) if l_match else None

    # Extract cylinder count
    cyl_match = re.search(r'(\d+) Cylinder', engine_desc)
    cylinders = int(cyl_match.group(1)) if cyl_match else None

    # Check if turbocharged
    turbocharged = 1 if 'Turbo' in engine_desc else 0

    return hp, displacement, cylinders, turbocharged

dt[['HP', 'Liters', 'Cylinders', 'Turbocharged']] = dt['engine'].apply(
    lambda x: pd.Series(extract_engine_features(x))
)
drop_columns = ['engine', 'model']
dt.drop(drop_columns, axis=1, inplace=True)

dt.fillna(dt.mean(), inplace=True)

dt.sample(15)

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,car_age,Luxurious,HP,Liters,Cylinders,Turbocharged
86578,275111,14,2006,2700,2,12,302,14,1,0,8.00000,0,310.000000,4.600000,8.000000,0.0
70514,259047,31,2014,95070,2,16,29,14,1,0,18.00000,0,181.000000,1.600000,4.000000,0.0
13194,201727,26,2013,70700,2,38,29,14,1,0,16.00000,0,375.000000,5.000000,8.000000,0.0
35248,223781,36,2020,6682,3,39,46,14,1,1,9.00000,0,342.797283,3.801503,6.372479,0.0
33189,221722,42,2014,61000,2,49,302,155,0,0,10.00000,0,330.000000,3.000000,6.000000,0.0
59788,248321,4,2008,125470,2,38,261,14,1,0,3.00000,0,360.000000,4.800000,8.000000,0.0
28107,216640,14,2016,230000,2,38,302,71,1,0,4.00000,0,365.000000,3.500000,6.000000,0.0
3233,191766,9,2010,100000,2,38,224,10,1,0,13.00000,0,210.000000,4.000000,6.000000,0.0
29031,217564,26,2015,92080,2,16,29,14,1,0,4.00000,0,240.000000,2.000000,4.000000,0.0
61661,250194,27,2010,136400,2,38,29,14,0,0,8.85398,1,272.000000,3.500000,6.000000,0.0


In [197]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            125690 non-null  int64  
 1   brand         125690 non-null  int64  
 2   model_year    125690 non-null  int64  
 3   milage        125690 non-null  int64  
 4   fuel_type     125690 non-null  int64  
 5   transmission  125690 non-null  int64  
 6   ext_col       125690 non-null  int64  
 7   int_col       125690 non-null  int64  
 8   accident      125690 non-null  int64  
 9   clean_title   125690 non-null  int64  
 10  car_age       125690 non-null  float64
 11  Luxurious     125690 non-null  int64  
 12  HP            125690 non-null  float64
 13  Liters        125690 non-null  float64
 14  Cylinders     125690 non-null  float64
 15  Turbocharged  125690 non-null  float64
dtypes: float64(5), int64(11)
memory usage: 15.3 MB


## Sample Submission File

In [198]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162610 entries, 0 to 188532
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            162610 non-null  int64  
 1   brand         162610 non-null  int64  
 2   model_year    162610 non-null  int64  
 3   milage        162610 non-null  int64  
 4   fuel_type     162610 non-null  int64  
 5   transmission  162610 non-null  int64  
 6   ext_col       162610 non-null  int64  
 7   int_col       162610 non-null  int64  
 8   accident      162610 non-null  int64  
 9   clean_title   162610 non-null  int64  
 10  price         162610 non-null  int64  
 11  car_age       162610 non-null  int64  
 12  Luxurious     162610 non-null  int64  
 13  HP            162610 non-null  float64
 14  Liters        162610 non-null  float64
 15  Cylinders     162610 non-null  float64
 16  Turbocharged  162610 non-null  float64
dtypes: float64(4), int64(13)
memory usage: 26.4 MB


In [200]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            125690 non-null  int64  
 1   brand         125690 non-null  int64  
 2   model_year    125690 non-null  int64  
 3   milage        125690 non-null  int64  
 4   fuel_type     125690 non-null  int64  
 5   transmission  125690 non-null  int64  
 6   ext_col       125690 non-null  int64  
 7   int_col       125690 non-null  int64  
 8   accident      125690 non-null  int64  
 9   clean_title   125690 non-null  int64  
 10  car_age       125690 non-null  float64
 11  Luxurious     125690 non-null  int64  
 12  HP            125690 non-null  float64
 13  Liters        125690 non-null  float64
 14  Cylinders     125690 non-null  float64
 15  Turbocharged  125690 non-null  float64
dtypes: float64(5), int64(11)
memory usage: 15.3 MB


In [201]:
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'class': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('results.csv', index=False)
print("Submission file created: results.csv")

Submission file created: submission_file.csv
